In [ ]:
import yaml
import pathlib
import anndata
import pandas as pd
from wmb import cemba, brain

In [ ]:
cluster_dir = '../MajorRegion/'

cluster_dir = pathlib.Path(cluster_dir).absolute()
cluster_dir.mkdir(exist_ok=True)

In [ ]:
def prepare_group(group, cells):
    group_dir = cluster_dir / group
    group_dir.mkdir(exist_ok=True)
    config_dir = group_dir / 'config'
    config_dir.mkdir(exist_ok=True)

    # subset cells
    with open(group_dir / 'select_cells.txt', 'w') as f:
        f.write('\n'.join(cells.index))

    for path in pathlib.Path('config/').glob('*.yaml'):
        with open(path) as f, open(config_dir / path.name, 'w') as outf:
            config = yaml.safe_load(f)

            # adjust the default parameters for all the groups
            if 'select_cells' in config:
                config['select_cells'] = 'select_cells.txt'
            if path.name == '06.yaml':
                # key parameter in clustering step
                config['leiden_resolution'] = 0.5
                config['target_accuracy'] = 0.9
                config['min_cluster_size'] = 30
                config['n_jobs'] = 20
            yaml.safe_dump(config, outf)
    return

In [ ]:
meta = cemba.get_mc_mapping_metric()
meta['MajorRegion'] = meta['DissectionRegion'].map(brain.map_cemba_id_to_major_region())

In [ ]:
meta['MajorRegion'].value_counts()

In [ ]:
for group, cells in meta.groupby('MajorRegion'):
    prepare_group(group, cells)

In [ ]:
# copy the snakefile
with open('../MajorRegion/Snakefile', 'w') as outf, open('Snakefile') as f:
    snakefile = f.read()
    snakefile = snakefile.replace('REPLACE_TEMPLATE_DIR',
                                  str(pathlib.Path().absolute()))
    outf.write(snakefile)